# Model

## Import and read Data

In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import os
from train_tabular import fit
columns = ['COR', 'FSH', 'FT4', 'IGF1', 'LH','PROL', 'TEST','Patient_age','Patient_gender']
os.environ["WANDB_SILENT"] = "true"

X_train = pd.read_csv(r'../../data/train_data_dirty.csv')
Y_train = X_train["Category"]

X_test = pd.read_csv(r'../../data/test_data_dirty.csv')
Y_test = X_test["Category"]
X_train= X_train[columns + ['fold']]
X_test= X_test[columns]

## Preprocessing

In [2]:
# define preprocessing function
X_train.loc[X_train['Patient_gender'] =='male','Patient_gender'] =1
X_train.loc[X_train['Patient_gender'] =='female','Patient_gender'] =0
X_test.loc[X_test['Patient_gender'] =='male','Patient_gender'] =1
X_test.loc[X_test['Patient_gender'] =='female','Patient_gender'] =0

## Setup Model

In [3]:
model = RandomForestClassifier(n_estimators=100,random_state=42)

In [4]:
wandbadd={'n_estimators':100,'random_state':42}

## Fit Model and Evaluate Model

In [5]:
models = []
for fold in range(0,5):
    fitted_model_fold = fit(model,X_train,Y_train,X_test,Y_test,fold,"Tab-Data-RandomForest-Dirty","RandomForest",None,wandb_additional_config=wandbadd)
    models.append(fitted_model_fold)

In [6]:
fitted_model_fold = fit(model,X_train,Y_train,X_test,Y_test,'all',"Tab-Data-RandomForest-All-Dirty","RandomForest",wandb_additional_config=wandbadd)